# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = pd.read_csv('weatherdata.csv')
weather_data.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Albany,42.6001,-73.9662,61.41,80,44,4.07,US,1632182283
1,1,Cabo San Lucas,22.8909,-109.9124,83.01,72,94,9.62,MX,1632182143
2,2,Sørland,67.6670,12.6934,46.54,86,100,23.13,NO,1632182283
3,3,Fare,-16.7000,-151.0167,79.05,69,12,13.22,PF,1632182284
4,4,Fairmont,39.4851,-80.1426,75.78,77,95,3.33,US,1632182217


In [3]:
gmaps.configure(api_key=g_key)

locations = weather_data[['Lat','Lng']]
humidities = weather_data['Humidity']


In [4]:
fig = gmaps.figure()

try:
    heat_layer = gmaps.heatmap_layer(locations, weights=humidities, 
                                     dissipating=False, max_intensity=100,
                                     point_radius = 2)
except:
    print('There was an error with the Google heatmap API call.')
    
fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Ideal Conditions
low = 72
high = 79
wind_speed_max = 10
cloudiness_max = 0

ideal_data = weather_data[(weather_data['Wind Speed'] <= wind_speed_max) & (weather_data['Cloudiness'] <= cloudiness_max) & (weather_data['Max Temp'] <= high)& (weather_data['Max Temp'] >= low)]
ideal_data = ideal_data.dropna()

ideal_data.head(20)





,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
25,25,Maceió,-9.6658,-35.7353,74.64,88,0,3.44,BR,1632182292
51,51,Naze,28.3667,129.4833,77.92,71,0,1.52,JP,1632182187
426,426,Cayenne,4.9333,-52.3333,78.84,89,0,0.00,GF,1632182570
480,480,Kiryat Gat,31.6100,34.7642,75.88,74,0,5.93,IL,1632182418


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = ideal_data
hotel_df['Hotel Name'] = 'error: none found'

target_type = 'lodging'
radius = 5000

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
for index, row in hotel_df.iterrows():
    location = f"{row['Lat']},{row['Lng']}"
    params ={
        'location':location,
        'type':target_type,
        'radius':radius,
        'key':g_key
    }
    
    
    response = requests.get(base_url, params).json()
    try:
        hotel_name = response["results"][0]["name"]
        print(f"City: {row['City']}")
        print(f'Nearest Hotel: {hotel_name}')
        print('________________________________________')
        hotel_df.loc[index, ['Hotel Name']] = hotel_name
    except:
        print(f"City: {row['City']}")
        print('error: No Hotel Found')
        print('________________________________________')
          





City: Maceió
Nearest Hotel: Poutur Pousada
________________________________________
City: Naze
Nearest Hotel: オーベルジュ・アガルイ
________________________________________
City: Cayenne
Nearest Hotel: Hôtel Le Dronmi
________________________________________
City: Kiryat Gat
Nearest Hotel: OlusHome
________________________________________


In [7]:
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
25,25,Maceió,-9.6658,-35.7353,74.64,88,0,3.44,BR,1632182292,Poutur Pousada
51,51,Naze,28.3667,129.4833,77.92,71,0,1.52,JP,1632182187,オーベルジュ・アガルイ
426,426,Cayenne,4.9333,-52.3333,78.84,89,0,0.00,GF,1632182570,Hôtel Le Dronmi
480,480,Kiryat Gat,31.6100,34.7642,75.88,74,0,5.93,IL,1632182418,OlusHome


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
# Create symbol layer
layer = gmaps.marker_layer(
    locations, 
    info_box_content=hotel_info
)

# Display figure

fig.add_layer(layer)

fig


Figure(layout=FigureLayout(height='420px'))